# Загрузка Pandas и очистка данных

In [97]:
import pandas as pd

In [98]:
df = pd.read_csv('main_task.csv')

In [99]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки
# df.Restaurant_id = df.Restaurant_id.apply(lambda x: int(x[3:]))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [100]:
newdf = df.drop(columns=['Restaurant_id','City','Cuisine Style','Price Range','Reviews','URL_TA','ID_TA'])
newdf.rename(columns={'Number of Reviews': 'review_num'}, inplace=True)
# newdf[newdf.review_num.isna() == False]
IQR = newdf.review_num.quantile(0.75) - newdf.review_num.quantile(0.25)
perc25 = newdf.review_num.quantile(0.25)
perc75 = newdf.review_num.quantile(0.75)
real_mean = newdf.review_num.loc[newdf.review_num.between(perc25 - 1.5*IQR, perc75 + 1.5*IQR)].mean()
df['Number of Reviews'] = newdf.review_num.fillna(real_mean)
df['Restaurant_id'] = df.Restaurant_id.str[3:]
df = df.drop(columns=['City','Cuisine Style','Price Range','Reviews','URL_TA','ID_TA'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   Ranking            40000 non-null  float64
 2   Rating             40000 non-null  float64
 3   Number of Reviews  40000 non-null  float64
dtypes: float64(3), object(1)
memory usage: 1.2+ MB


# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [101]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'Rating'], axis = 1)
y = df['Rating']

In [102]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [103]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [104]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [105]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [106]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.43638770734126986
